In [54]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
uciml_sms_spam_collection_dataset_path = kagglehub.dataset_download('uciml/sms-spam-collection-dataset')

print('Data source import complete.')


Using Colab cache for faster access to the 'sms-spam-collection-dataset' dataset.
Data source import complete.


In [55]:
import string

import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [56]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [57]:
df = pd.read_csv(f'{uciml_sms_spam_collection_dataset_path}/spam.csv', encoding='latin1')

In [58]:
display(df.head())

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [59]:
df.columns

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')

In [60]:
df = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv', encoding='latin1')
df['v2'] = df['v2'].apply(lambda x: str(x).replace('\r\n', ''))

In [61]:
display(df.head())

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [62]:
df.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [63]:
df.info

<bound method DataFrame.info of         v1                                                 v2 Unnamed: 2  \
0      ham  Go until jurong point, crazy.. Available only ...        NaN   
1      ham                      Ok lar... Joking wif u oni...        NaN   
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...        NaN   
3      ham  U dun say so early hor... U c already then say...        NaN   
4      ham  Nah I don't think he goes to usf, he lives aro...        NaN   
...    ...                                                ...        ...   
5567  spam  This is the 2nd time we have tried 2 contact u...        NaN   
5568   ham              Will Ì_ b going to esplanade fr home?        NaN   
5569   ham  Pity, * was in mood for that. So...any other s...        NaN   
5570   ham  The guy did some bitching but I acted like i'd...        NaN   
5571   ham                         Rofl. Its true to its name        NaN   

     Unnamed: 3 Unnamed: 4  
0           NaN        NaN  
1           NaN        NaN  
2           NaN        NaN  
3           NaN        NaN  
4           NaN        NaN  
...         ...        ...  
5567        NaN        NaN  
5568        NaN        NaN  
5569        NaN        NaN  
5570        NaN        NaN  
5571        NaN        NaN  

[5572 rows x 5 columns]>

In [64]:
import pandas as pd
import string
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from nltk.stem import PorterStemmer

# make sure df exists already
print(df.columns)   # just to confirm

ps = PorterStemmer()
corpus = []

# use sklearn stopwords (works offline)
stopwords_set = set(ENGLISH_STOP_WORDS)

for text in df['v2']:
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation)).split()
    text = [ps.stem(word) for word in text if word not in stopwords_set]
    text = ' '.join(text)
    corpus.append(text)

print("Corpus created successfully!")
print("Example:", corpus[0][:200])

Index(['v1', 'v2', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], dtype='object')
Corpus created successfully!
Example: jurong point crazi avail bugi n great world la e buffet cine got amor wat


In [65]:
df['v2'].iloc[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [66]:
corpus[0]

'jurong point crazi avail bugi n great world la e buffet cine got amor wat'

In [67]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# Create Bag of Words
vectorizer = CountVectorizer(max_features=5000)  # limit features for better performance
X = vectorizer.fit_transform(corpus).toarray()

# Convert label column to numbers (ham = 0, spam = 1)
df['label_num'] = df['v1'].map({'ham': 0, 'spam': 1})
y = df['label_num']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [68]:
X[0]

array([0, 0, 0, ..., 0, 0, 0])

In [69]:
clf = RandomForestClassifier(n_jobs=-1)

clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [70]:
clf.score(X_test, y_test)

0.9730941704035875

In [71]:
df = df[['v1', 'v2']]

In [85]:
df = df.reset_index(drop=True)
# Changing to a known spam email from the dataset to demonstrate spam prediction
email_to_classify = df['v2'][2] # This is a spam email: 'Free entry in 2 a wkly comp to win FA Cup fina...'
print(email_to_classify)

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [73]:
print(email_to_classify)

I'm gonna be home soon and i don't want to talk about this stuff anymore tonight, k? I've cried enough today.


In [74]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

email_text = email_to_classify.lower().translate(str.maketrans('', '', string.punctuation)).split()
email_text = [stemmer.stem(word) for word in text if word not in stopwords_set]
email_text =' '.join(email_text)

email_corpus = [email_text]

X_email = vectorizer.transform(email_corpus)

In [75]:
clf.predict(X_email)

array([0])

In [76]:
df.columns

Index(['v1', 'v2'], dtype='object')

In [77]:
df['label_num'] = df['v1'].map({'ham': 0, 'spam': 1})

In [78]:
df.head()

,v1,v2,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [79]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=df)

MessageError: Error: credential propagation was unsuccessful

In [106]:
df.label_num.iloc[3]

np.int64(0)

In [105]:
result = df.label_num.iloc[3]   # your prediction

if int(result) == 1:
    print("Email is Spam ❌")
else:
    print("Email is Genuine ✅")


Email is Genuine ✅
